# Final Project - Finetuning User

In [1]:
!pip install transformers==4.57.1
!pip install trl==0.24.0
!pip install peft==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 108.3 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import transformers, trl, peft

2025-12-11 10:33:49.131147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765449229.290604      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765449229.336221      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# imports
import numpy as np 
import json
import os
import shutil
import subprocess
import sys
from typing import List
from datasets import Dataset


from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, pipeline
#import transformers, trl, peft
import torch
import random
torch.manual_seed(3407); random.seed(3407); np.random.seed(3407)


from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

## Get all the data

In [4]:
def setup_repo(repo_url: str, repo_name: str, work_dir: str = "/kaggle/working"):
    os.chdir(work_dir)
    
    # Remove repo if it exists
    if os.path.exists(os.path.join(work_dir, repo_name)):
        shutil.rmtree(os.path.join(work_dir, repo_name))
    
    # Clone repo
    subprocess.run(["git", "clone", repo_url], check=True)
    
    # Move into repo/data
    os.chdir(os.path.join(repo_name, "data"))


setup_repo("https://github.com/lkra/dstc11-track5.git", "dstc11-track5")


Cloning into 'dstc11-track5'...


In [5]:
## List all files in the current directory iteratively:
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./output_schema.json
./knowledge.json
./knowledge_aug_domain_reviews.json
./README.md
./knowledge_aug_reviews.json
./val/logs.json
./val/labels.json
./test/logs.json
./test/labels.json
./train/logs.json
./train/labels.json
./train/logs_bkp.json
./train/bkp/logs.json
./train/bkp/labels.json


In [6]:
with open('train/logs.json', 'r') as f:
    train_ds=json.load(f)

with open('train/labels.json', 'r') as f:
    labels=json.load(f)

with open('knowledge.json', 'r') as f:
    knowledge_base=json.load(f)

In [7]:
def format_dialogue(dialogue: List[dict]) -> List[dict]: 
    """
    Args:
    dialogue (List[dict]): A list of dictionaries where each dictionary contains two keys:
        - 'speaker' (str): A string indicating the speaker of the turn ('U' for user, 'S' for system).
        - 'text' (str): The text spoken by the respective speaker.

    Returns:
        List[dict]: A new array with a specific role and content

    """
    # Your solution here
    messages=[]
    messages.append({"role": "system", "content": "You are a user simulator."})
    for dialogue_element in dialogue:
        role = "assistant" if dialogue_element['speaker'] == 'U' else "user" #roles swapped so it learns to behave like a user
        messages.append({"role": role, "content": dialogue_element['text']})

    return messages

In [8]:
'''
def reformat_dataset(dataset, labels_dataset): 
    reformatted_dataset = {
        "messages": []
    }
    for sample_index in range(len(dataset)): 
        # Your solution here
        try:
            sample_dialogue = format_dialogue(dataset[sample_index])
            sample_response = labels_dataset[sample_index]['response']
            sample_dialogue.append({"role": "assistant", "content": sample_response})
            
            reformatted_dataset["messages"].append(sample_dialogue)
        except:
            continue


        
    return reformatted_dataset

reformatted_dataset = reformat_dataset(train_ds, labels)
dataset = Dataset.from_dict(reformatted_dataset)
dataset
'''

'\ndef reformat_dataset(dataset, labels_dataset): \n    reformatted_dataset = {\n        "messages": []\n    }\n    for sample_index in range(len(dataset)): \n        # Your solution here\n        try:\n            sample_dialogue = format_dialogue(dataset[sample_index])\n            sample_response = labels_dataset[sample_index][\'response\']\n            sample_dialogue.append({"role": "assistant", "content": sample_response})\n            \n            reformatted_dataset["messages"].append(sample_dialogue)\n        except:\n            continue\n\n\n        \n    return reformatted_dataset\n\nreformatted_dataset = reformat_dataset(train_ds, labels)\ndataset = Dataset.from_dict(reformatted_dataset)\ndataset\n'

In [9]:
def reformat_dataset(dataset): 
    reformatted_dataset = {
        "messages": []
    }
    for sample_index in range(len(dataset)):
    #for sample_index in range(1):
        try:
            sample_dialogue = format_dialogue(dataset[sample_index][:-1]) # exclude last user message so system learns to respond as a user
            sample_response = dataset[sample_index][-1]['text'] #use original last user message as response
            sample_dialogue.append({"role": "assistant", "content": sample_response})
            
            reformatted_dataset["messages"].append(sample_dialogue)
        except:
            continue

    return reformatted_dataset


reformatted_dataset = reformat_dataset(train_ds)
dataset = Dataset.from_dict(reformatted_dataset)
dataset

Dataset({
    features: ['messages'],
    num_rows: 32604
})

In [10]:
def process_dataset_split(split: str) -> Dataset: 
    """Loads, reformats, and processes a dataset split for model training or evaluation.

    This function loads a dataset split (e.g., 'val', 'test') and generates a dataset for it, similar to what we had for the train split.

    Args:
        split (str): The name of the dataset split to process

    Returns:
        dataset: A HuggingFace `Dataset` object that contains the preprocessed and reformatted data for the specified split.

    """
    with open(f'{split}/logs.json', 'r') as f:
        data=json.load(f)

    data_ds = reformat_dataset(data)
    new_dataset = Dataset.from_dict(data_ds)
    
    return new_dataset
    

validation_ds = process_dataset_split("val")
test_ds = process_dataset_split("test")

validation_ds, test_ds

(Dataset({
     features: ['messages'],
     num_rows: 4173
 }),
 Dataset({
     features: ['messages'],
     num_rows: 5475
 }))

In [11]:
model_id = "Qwen/Qwen3-1.7B"
tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
base = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [12]:
peft_cfg = LoraConfig(r=16, 
           lora_alpha=32, 
           lora_dropout = 0.05, 
           bias = "none", 
           use_rslora = False, 
           target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"])

model = get_peft_model(base, peft_cfg)

In [13]:
def pick_bf16():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return major >= 8
    return False

## Create agent 1: Assistant 1

In [14]:
NUM_TRAIN_EPOCHS = 1
LEARNING_RATE    = 7e-5
WARMUP_STEPS     = ((2113 * NUM_TRAIN_EPOCHS)//100) * 10

os.environ["WANDB_DISABLED"] = "true"

sft_args = SFTConfig(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    logging_steps=10,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    max_length=1024,
    optim="adamw_torch_fused",
    fp16=not pick_bf16(),
    bf16=pick_bf16(),
    packing=False,
    dataset_num_proc=2,
    report_to="none",
    seed=3407
)

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,      
    eval_dataset=validation_ds,
    args=sft_args,
    processing_class=tok
)

Tokenizing train dataset (num_proc=2):   0%|          | 0/32604 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/32604 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/4173 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/4173 [00:00<?, ? examples/s]

In [16]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,6.316900
20,5.936600
30,5.217200
40,4.086000
50,3.408700
60,2.753800
70,2.397200
80,2.117300
90,1.946000
100,1.866300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

TrainOutput(global_step=4076, training_loss=1.1825953711471333, metrics={'train_runtime': 10676.8617, 'train_samples_per_second': 3.054, 'train_steps_per_second': 0.382, 'total_flos': 6.926178323946701e+16, 'train_loss': 1.1825953711471333, 'entropy': 1.045391770926389, 'num_tokens': 6070299.0, 'mean_token_accuracy': 0.7216480536894365, 'epoch': 1.0})

In [17]:
trainer.save_model("outputs/adapterUser")  
tok.save_pretrained("outputs/adapterUser")

('outputs/adapterUser/tokenizer_config.json',
 'outputs/adapterUser/special_tokens_map.json',
 'outputs/adapterUser/chat_template.jinja',
 'outputs/adapterUser/vocab.json',
 'outputs/adapterUser/merges.txt',
 'outputs/adapterUser/added_tokens.json',
 'outputs/adapterUser/tokenizer.json')

In [18]:
!zip -r "finetunedUser.zip" "outputs/adapterUser"

  adding: outputs/adapterUser/ (stored 0%)
  adding: outputs/adapterUser/vocab.json (deflated 61%)
  adding: outputs/adapterUser/training_args.bin (deflated 51%)
  adding: outputs/adapterUser/special_tokens_map.json (deflated 69%)
  adding: outputs/adapterUser/adapter_config.json (deflated 57%)
  adding: outputs/adapterUser/tokenizer.json (deflated 81%)
  adding: outputs/adapterUser/adapter_model.safetensors (deflated 8%)
  adding: outputs/adapterUser/README.md (deflated 66%)
  adding: outputs/adapterUser/merges.txt (deflated 57%)
  adding: outputs/adapterUser/chat_template.jinja (deflated 76%)
  adding: outputs/adapterUser/added_tokens.json (deflated 68%)
  adding: outputs/adapterUser/tokenizer_config.json (deflated 90%)
